In [4]:
c=open('C:\\Users\jay.timbadia\Downloads\Multiutility_lrclassifier.txt', encoding='utf-8', errors='ignore')
import pandas as pd
import re
p = c.readlines()
# print(p)
y = p[0].split('LRRetain ')
print(y[1])
# t = re.findall(r'(LRRetain|LRReview)', p)
dataf = pd.DataFrame()
for i in p:
    t = re.findall(r'(LRRetain|LRReview)', i)
    if t[0] == 'LRReview':
        o = i.split('LRReview ')
        sentence = o[1]
        label = 'LRReview'
        dataf = dataf.append({'Sentence': sentence, 'Labels': label}, ignore_index=True)
        
    elif t[0] == 'LRRetain':
        o = i.split('LRRetain ')
        sentence = o[1]
        label = 'LRRetain'
        dataf = dataf.append({'Sentence': sentence, 'Labels': label}, ignore_index=True)
        
dataf['Labels'] = dataf['Labels'].map({'LRReview':1, 'LRRetain':0})

lab = [1, 0]
dataf.head()

Handelsblatt reported, earlier this week that Deutsche Telekom is having Board level discussions regarding an all-stock deal to merge T-Mobile with Sprint.



,Labels,Sentence
0,0,"Handelsblatt reported, earlier this week that ..."
1,0,"Handelsblatt a German publication reported, ea..."
2,0,"The WSJ reported, the ELM will be located in t..."
3,0,The deal is expected to close by early 2Q17 an...
4,0,"We believe, however, that before TMUS can shut..."


In [37]:
import collections
def bag_of_words(words):
    return dict([(word, True) for word in words])
def label_feats_from_corpus(dataf, feature_detector=bag_of_words):
    label_feats = collections.defaultdict(list)
    for index, row in dataf.iterrows():
        if row['Labels'] == 0:
            x = row['Sentence']
            re.split(r'[,;]', x)
            t = re.split(r'[, ]', x)
            space = ['']
            final_sent = list(set(t) - set(space))
            feats = feature_detector(final_sent)
            label_feats[0].append(feats)
        else:
            x = row['Sentence']
            re.split(r'[,;]', x)
            t = re.split(r'[, ]', x)
            space = ['']
            final_sent = list(set(t) - set(space))
            feats = feature_detector(final_sent)
            label_feats[1].append(feats)
    return label_feats
lfeats = label_feats_from_corpus(dataf)

2

In [38]:
l = 'jay is a, good bot'

import re
re.split(r'[,;]', l)
t = re.split(r'[, ]', l)
li = ['', 'fdf', '']
k = ['']
q = list(set(li) - set(k))
t

defaultdict(list,
            {0: [{'to': True,
               'regarding': True,
               'with': True,
               'discussions': True,
               'an': True,
               'this': True,
               'deal': True,
               'Deutsche': True,
               'Handelsblatt': True,
               'is': True,
               'Sprint.\n': True,
               'reported': True,
               'Board': True,
               'all-stock': True,
               'having': True,
               'earlier': True,
               'T-Mobile': True,
               'merge': True,
               'level': True,
               'Telekom': True,
               'that': True,
               'week': True},
              {'to': True,
               'regarding': True,
               'with': True,
               'German': True,
               'discussions': True,
               'publication': True,
               'an': True,
               'this': True,
               'deal': True,
               

In [43]:
def split_label_feats(lfeats, split=0.75):
    train_feats = []
    test_feats = []
    for label, feats in lfeats.items():
        cutoff = int(len(feats) * split)
        train_feats.extend([(feat, label) for feat in feats[:cutoff]])
        test_feats.extend([(feat, label) for feat in feats[cutoff:]])
    return train_feats, test_feats
train_feats, test_feats = split_label_feats(lfeats)

({'to': True,
  'regarding': True,
  'with': True,
  'discussions': True,
  'an': True,
  'this': True,
  'deal': True,
  'Deutsche': True,
  'Handelsblatt': True,
  'is': True,
  'Sprint.\n': True,
  'reported': True,
  'Board': True,
  'all-stock': True,
  'having': True,
  'earlier': True,
  'T-Mobile': True,
  'merge': True,
  'level': True,
  'Telekom': True,
  'that': True,
  'week': True},
 0)

In [40]:
from nltk.classify import NaiveBayesClassifier
nb_classifier = NaiveBayesClassifier.train(train_feats)
from nltk.classify.util import accuracy
accuracy(nb_classifier, test_feats)

0.7359550561797753

In [42]:
from nltk.classify import MaxentClassifier
me_classifier = MaxentClassifier.train(train_feats, algorithm='iis', trace=0, max_iter=1, min_lldelta=0.5)
accuracy(me_classifier, test_feats)

0.7359550561797753